In [ ]:
import os
import glob
import subprocess

drive = 'seagate'  # 'passport' or 'seagate'

raw_files_path = '/Volumes/Seagate_storage/Foraging_study_copy2/' if drive=='seagate' else '/Volumes/MyPassport/Hailey_data_preproc/fMRI_data/'
scan_path = '/Volumes/Seagate_storage/Foraging_study_copy2/preprocess/' if drive=='seagate' else '/Volumes/MyPassport/Hailey_data_preproc/fMRI_data/preprocess/'
exclude = ['sub121']
all_subj_dir = [x for x in glob.glob(scan_path + 'sub*') if not any(j in x for j in exclude)]
use_sb_epi = True  # Use scout file as initial high res?

for subj_path in all_subj_dir[0:8]:  # subj_path=all_subj_dir[5]
    subj_name = subj_path.split('/')[-1]
    
    print("\n--------Creating fsf for subject", subj_name)
    output_dir = subj_path + '/L1stats_10Mar2022_tempDerivs_noMotionOutliers'
    save_fsf_folder = raw_files_path+subj_name+'/regressors_23Feb2022/'

    ########################## Find files ##########################
    func_scan = glob.glob(raw_files_path+subj_name+'/preproc_scans_16Feb2022/*epiRun.nii*')[0]
    struct = glob.glob(raw_files_path+subj_name+'/preproc_scans_16Feb2022/*structural_brain*.nii*')[0]
    subj_feat_dir = glob.glob(subj_path + '/*.feat')[0]
    
    ########################## Edit pnm output paths #####################
    pnm_root = raw_files_path + subj_name + '/pnm_output/'
    pnm_path = pnm_root + 'pnm_evlist.txt'
#     if drive == 'passport':
#         pnm_root_to_replace = '/Volumes/Seagate_storage/Foraging_study_copy2/'+subj_name+'/pnm_output/'
#         with open(pnm_path) as file:
#             evlist = file.readlines()

#         for l in range(len(evlist)):
#             if pnm_root_to_replace in evlist[l]:
#                 evlist[l] = evlist[l].replace(pnm_root_to_replace, pnm_root)

#         with open(pnm_root + 'pnm_evlist_passport.txt', 'w') as f:
#             for item in evlist:
#                 f.write("%s" % item)

    ########################## Create & edit L1 stats fsf #####################
    # Open template file
    L1_template_path = raw_files_path+'/fsf_templates/L1stats_10Mar2022_tempDerivs_motionOutliers.fsf'
    with open(L1_template_path) as file:
        L1_template = file.readlines()
        
    # Get number of volumes from the EPI using fslval
    process = subprocess.Popen(['fslval', func_scan, 'dim4'],
                          stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    epi_vols, stderr = process.communicate()
    epi_vols = epi_vols.decode('utf-8').strip(' \n')
    nvol_ind = [x for x in range(len(L1_template)) if 'set fmri(npts)' in L1_template[x]][0]
    L1_template[nvol_ind] = 'set fmri(npts) '+ epi_vols + '\n'
    
    # Get number of voxels from the EPI using fslval
    process = subprocess.Popen(['fslstats', func_scan, '-v'],
                          stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    epi_vox, stderr = process.communicate()
    epi_vox = epi_vox.decode('utf-8').split(' ')[0]
    totalvox_ind = [x for x in range(len(L1_template)) if 'set fmri(totalVoxels)' in L1_template[x]][0]
    L1_template[totalvox_ind] = 'set fmri(totalVoxels) '+epi_vox+'\n'

    # Replace path to filtered_func_data.nii.gz
    featdir_ind =  [x for x in range(len(L1_template)) if 'set feat_files(1)' in L1_template[x]][0]
    L1_template[featdir_ind] = 'set feat_files(1) "'+subj_feat_dir+'/filtered_func_data.nii.gz"\n'
    
    # Replace path to struct
    struct_ind = [x for x in range(len(L1_template)) if 'set highres_files(1)' in L1_template[x]][0]
    L1_template[struct_ind] = 'set highres_files(1) "'+struct+'"\n'
    
    # Replace output directory
    output_ind = [x for x in range(len(L1_template)) if 'set fmri(outputdir)' in L1_template[x]][0]
    L1_template[output_ind] = 'set fmri(outputdir) "'+output_dir+'"\n'
    
    # Replace pnm output paths
    pnm_ind = [x for x in range(len(L1_template)) if 'set fmri(motionevsbeta)' in L1_template[x]][0]
    pnm_path_to_set = pnm_path if drive=='seagate' else pnm_root + 'pnm_evlist_passport.txt'
    L1_template[pnm_ind] = 'set fmri(motionevsbeta) "'+pnm_path_to_set+'"\n'
    
    # Make all paths specific to this subject
    to_replace = 'sub109'
    for line in range(len(L1_template)):
        if to_replace in L1_template[line]:
            L1_template[line] = L1_template[line].replace(to_replace, subj_name)
    
    ## Change regressors directory
    #to_replace = '/Users/haileytrier/Desktop/regressors_23Feb2022/'
    #replace_with = '/Volumes/Seagate_storage/Foraging_study_copy2/'+subj_name+'/regressors_23Feb2022/'
    #for line in range(len(L1_template)):
        #if to_replace in L1_template[line]:
            #L1_template[line] = L1_template[line].replace(to_replace, replace_with)
    
    # Replace paths from other hard drive
    other_drive_path = '/Volumes/Seagate_storage/Foraging_study_copy2/' if drive=='passport' else '/Volumes/MyPassport/Hailey_data_preproc/fMRI_data/'
    current_drive_path = raw_files_path
    for line in range(len(L1_template)):
        if other_drive_path in L1_template[line]:
            L1_template[line] = L1_template[line].replace(other_drive_path, current_drive_path)
            
    # Write to first level file
    with open(save_fsf_folder+'L1stats_10Mar2022_tempDerivs_noMotionOutliers_'+drive+'.fsf', 'w') as f:
        for item in L1_template:
            f.write("%s" % item)